In [96]:
import pandas as pd
import numpy as np

In [97]:
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import leaguegamefinder

In [98]:
gamefinder = leaguegamefinder.LeagueGameFinder(season_type_nullable='Playoffs',league_id_nullable='00')
games = gamefinder.get_data_frames()[0]

In [205]:
gamefinder = leaguegamefinder.LeagueGameFinder(season_type_nullable='Playoffs',league_id_nullable='00')
games = gamefinder.get_data_frames()
games[0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42021,1610612738,BOS,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,239,90,...,0.917,11,30,41,27,8,8,22,16,-13.0
1,42021,1610612744,GSW,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,241,103,...,1.000,15,29,44,27,13,7,15,20,13.0
2,42021,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,238,94,...,0.677,8,39,47,18,2,2,18,16,-10.0
3,42021,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,240,104,...,0.867,4,35,39,23,9,2,6,28,10.0
4,42021,1610612744,GSW,Golden State Warriors,0042100404,2022-06-10,GSW @ BOS,W,241,107,...,0.800,16,39,55,20,10,5,16,21,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,41983,1610612765,DET,Detroit Pistons,0048300004,1984-04-17,DET vs. NYK,L,240,93,...,0.731,12,32,44,20,11,4,18,19,NaN
6076,41983,1610612737,ATL,Atlanta Hawks,0048300005,1984-04-17,ATL @ MIL,L,240,89,...,0.707,13,27,40,20,9,7,18,14,NaN
6077,41983,1610612749,MIL,Milwaukee Bucks,0048300005,1984-04-17,MIL vs. ATL,W,240,105,...,0.696,7,33,40,28,11,6,20,28,NaN
6078,41983,1610612743,DEN,Denver Nuggets,0048300003,1984-04-17,DEN @ UTH,L,240,121,...,0.769,14,32,46,28,10,1,19,26,NaN


In [99]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(games)

In [6]:
playoff_21 = game_df[game_df['GAME_DATE'] > "2022-01-01"]

In [100]:
game_list = playoff_21['GAME_ID'].to_list()

In [104]:
full = game_df['GAME_ID'].to_list()

In [118]:
all = []

for game in full:
    gamelogs3 = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game).player_stats
    game_log3 = gamelogs3.get_data_frame()
    all.append(game_log3)

all[1]

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [28]:
player_box_score_all = pd.concat(gamess)

In [29]:
player_box_score_all.to_csv('/Users/amos/Downloads/all_players_boxscore_2022.csv')

In [58]:
ee = game_df[game_df['GAME_DATE'] > "2022-01-01"]

In [73]:
finals_games = game_df[game_df['GAME_ID'].str.contains('.....004..',regex=True)]

In [84]:
finals_games.head()

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,...,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
2,42021,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,240,104,...,0.677,8,39,47,18,2,2,18,16,-10.0
6,42021,1610612738,BOS,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,241,97,...,0.800,16,39,55,20,10,5,16,21,10.0
9,42021,1610612738,BOS,Boston Celtics,0042100403,2022-06-08,BOS vs. GSW,W,240,116,...,0.867,6,25,31,22,7,5,16,23,-16.0
13,42021,1610612744,GSW,Golden State Warriors,0042100402,2022-06-05,GSW vs. BOS,W,241,107,...,0.765,6,37,43,24,5,7,18,18,-19.0
18,42021,1610612744,GSW,Golden State Warriors,0042100401,2022-06-02,GSW vs. BOS,L,241,108,...,0.813,7,32,39,33,7,6,12,13,12.0


In [87]:
finals_games.to_csv('/Users/amos/Downloads/finals.csv')

In [88]:
game_list_finals = finals_games['GAME_ID'].to_list()

In [89]:
finalss = []

for game in game_list_finals:
    gamelogs2 = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game).player_stats
    game_log2 = gamelogs2.get_data_frame()
    finalss.append(game_log2)

finalss[1]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0042100404,1610612744,GSW,Golden State,203952,Andrew Wiggins,Andrew,F,,43:26,...,3.0,13.0,16.0,2.0,1.0,0.0,2.0,2.0,17.0,20.0
1,0042100404,1610612744,GSW,Golden State,203490,Otto Porter Jr.,Otto,F,,14:30,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,-1.0
2,0042100404,1610612744,GSW,Golden State,203110,Draymond Green,Draymond,C,,32:58,...,5.0,4.0,9.0,8.0,4.0,0.0,2.0,3.0,2.0,0.0
3,0042100404,1610612744,GSW,Golden State,202691,Klay Thompson,Klay,G,,40:49,...,1.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,18.0,2.0
4,0042100404,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,40:41,...,2.0,8.0,10.0,4.0,0.0,0.0,5.0,3.0,43.0,11.0
5,0042100404,1610612744,GSW,Golden State,1626172,Kevon Looney,Kevon,,,28:10,...,4.0,7.0,11.0,2.0,1.0,1.0,3.0,3.0,6.0,21.0
6,0042100404,1610612744,GSW,Golden State,1627780,Gary Payton II,Gary,,,9:54,...,1.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,5.0,-1.0
7,0042100404,1610612744,GSW,Golden State,1629673,Jordan Poole,Jordan,,,20:33,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,14.0,2.0
8,0042100404,1610612744,GSW,Golden State,202357,Nemanja Bjelica,Nemanja,,,9:00,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,-4.0
9,0042100404,1610612744,GSW,Golden State,2738,Andre Iguodala,Andre,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
finals_player_box_score = pd.concat(finalss)

In [92]:
finals_player_box_score.to_csv('/Users/amos/Downloads/finals_player_box_score.csv')